# Algorithm refactoring

## Layout

```
QuICT/algorithm/quantum_algorithm/
├── __init__.py
├── grover
│   ├── __init__.py
│   ├── grover.py
│   ├── partial_grover.py
│   └── search_with_prior_knowledge.py
└── shor
    ├── __init__.py
    ├── BEA.py
    ├── BEA_zip.py
    ├── HRS.py
    ├── HRS_zip.py
    ├── shor_factor.py
    └── utility.py
```

In [15]:
from QuICT.core.gate import *
from QuICT.simulation.gpu_simulator import ConstantStateVectorSimulator

import random
random.seed(2022)

## Grover Module

- oracle ancilla support
- bug fix: use `MCTOneAux` instead of `MCTLinearOneDirtyAux` to avoid unexpected behaviour

### example

To start with, we construct the oracle $O\ket{x}\ket{-}\ket{0}=(-1)^{f(x)}\ket{x}\ket{-}\ket{0}$ which use 2 ancillary qubits:

In [16]:
from QuICT.algorithm.quantum_algorithm.grover import Grover,PartialGrover
from QuICT.qcda.synthesis.mct import MCTOneAux

def main_oracle(n, f):
    assert len(f) == 1, "only 1 target support for this oracle"
    assert f[0] >= 0 and f[0] < (1 << n)
    result_q = [n]
    cgate = CompositeGate()
    target_binary = bin(f[0])[2:].rjust(n, "0")
    with cgate:
        # |-> in result_q
        X & result_q[0]
        H & result_q[0]
        # prepare for MCT
        for i in range(n):
            if target_binary[i] == "0":
                X & i
    MCTOneAux.execute(n + 2) | cgate
    # un-compute
    with cgate:
        for i in range(n):
            if target_binary[i] == "0":
                X & i
        H & result_q[0]
        X & result_q[0]
    return 2, cgate

Construct Grover search circuit on oracle with $f(x)=[x=target]$ and run:

In [17]:
n = 10
print(f"run with n = {n}")
target = random.randrange(0,1<<n)
k, oracle = main_oracle(n, [target])
circ = Grover.circuit(n, k, oracle)

ConstantStateVectorSimulator().run(circ)
result = int(circ[list(range(n))])
print(f"target = {target}")
print(f"result = {result}")

run with n = 10
target = 591
result = 591


Construct Partial Grover search circuit on oracle with $f(x)=[x=target]$ and run:

In [18]:
n = 10
n_block = 3
print(f"run with n = {n}, block size = {n_block}")
target = random.randrange(0,1<<n)
k, oracle = main_oracle(n, [target])
circ = PartialGrover.circuit(n, n_block, k, oracle)

ConstantStateVectorSimulator().run(circ)
result = int(circ[list(range(n))])
print(f"target in block {bin(target)[2:2+n_block]}")
print(f"result in block {bin(result)[2:2+n_block]}")

run with n = 10, block size = 3
target in block 100
result in block 100


## Shor Module

-  `circuit` method support by using `Trigger` operator
- bug fix: little-endian IQFT/QFT in arithmetic circuit and big-endian IQFT/QFT in QPE circuit
- bug fix: use $CU1(\theta) = \begin{bmatrix}
 1 & 0 & 0 & 0\\ 
 0 & 1 & 0 & 0\\ 
 0 & 0 & 1 & 0\\ 
 0 & 0 & 0 & e^{i\theta}
\end{bmatrix}$ instead of
$CR_z(\theta) = \begin{bmatrix}
 1 & 0 & 0 & 0\\ 
 0 & 1 & 0 & 0\\ 
 0 & 0 & e^{-i\theta/2} & 0\\ 
 0 & 0 & 0 & e^{i\theta/2}
\end{bmatrix}$ in QFT/IQFT

The quantum part of Shor's factoring algorithm is the order-finding circuit, which can be split into Quantum Phase Estimation (QPE) circuit and modular exponentiation circuit. By using two different constructions of QPE circuit (with or without the one-qubit-trick) and two different constructions of modular exponentiation circuit (in `QuICT.qcda.synthesis.arithmetic.bea` and `QuICT.qcda.synthesis.arithmetic.hrs`), four implementations of Shor's factoring algorithm is given. 

### example

Use `ShorFactor(mode, N).run()` to factor a number $N$:

In [19]:
from QuICT.algorithm.quantum_algorithm import ShorFactor

input  = 35
sf = ShorFactor(mode="BEA_zip", N=input)
circ, indices = sf.circuit()

output = sf.run(simulator=ConstantStateVectorSimulator(), circuit=circ, indices=indices)
print(f"input  = {input}")
print(f"output = {output}")

input  = 35
output = 5


More specificly, the order-finding circuit will run serval times and the output will be least common multiple of results.
```
>>> output = ShorFactor(mode="BEA_zip", N=21).run(simulator=ConstantStateVectorSimulator())
INFO:root:      circuit construction begin: circuit: n = 5 t = 13
INFO:root:round = 0
INFO:root:Quantumly determine the order of the randomly chosen a = 11
INFO:root:      order_finding begin: circuit: n = 5 t = 13
INFO:root:      phi~ (approximately s/r) in decimal form is 0.1666259765625
INFO:root:      Continued fraction expansion of phi~ is 1/6
INFO:root:      success!
INFO:root:      order_finding begin: circuit: n = 5 t = 13
INFO:root:      phi~ (approximately s/r) in decimal form is 0.3333740234375
INFO:root:      Continued fraction expansion of phi~ is 1/3
INFO:root:      order_finding begin: circuit: n = 5 t = 13
INFO:root:      phi~ (approximately s/r) in decimal form is 0.33349609375
INFO:root:      Continued fraction expansion of phi~ is 1/3
INFO:root:Shor succeed: found factor 7, with the help of a = 11, r = 6
```

Not recomended though, one can construct order-finding circuit and then run it on $(p, N)$ where $gcd(p, N)=1$:

In [20]:
from fractions import Fraction
from math import gcd
from QuICT.algorithm.quantum_algorithm.shor.BEA_zip import construct_circuit as BEA_construct_circuit

def naive_order_finding(a, N):
    for i in range(1, N):
        if (a ** i) % N == 1:
            return i
    return 0

N = 35
simulator = ConstantStateVectorSimulator()
p = random.choice(list(filter(lambda x: gcd(x, N) == 1, list(range(N)))))
print(f"testing ({p:2},{N:2})...")
circ, indices = BEA_construct_circuit(p, N)
simulator.run(circ)
phi = eval("0b" + "".join([str(trig.measured[0]) for trig in indices])) / (
    1 << len(indices)
)
print(f"phi = {phi:.5f}")
result = Fraction(phi).limit_denominator(N - 1).denominator
print(f"target = {naive_order_finding(p, N)}")
print(f"result = {result}")

testing (22,35)...
phi = 0.25000
target = 4
result = 4
